## Criar um ambiente virtual e instalar todos os pacotes Python, via pip install.
    Aqui é assumido que o python e o gerenciador de pacotes pip já estão 
    instalados e funcionando em seu ambiente.


### Criando um ambiente virtual:
    python -m venv .venv

### Ativando o ambiente virtual Python:
    source .venv/bin/activate

### Os pacotes Python, podem ser instalados usando com requirements.txt, na mesma pasta do script:
    pip install -r requirements.txt

## Importar bibliotecas que foram instaladas via pip


In [13]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.point import Point
from cryptography.fernet import Fernet
from datetime import datetime
from google.oauth2.service_account import Credentials
from google.oauth2 import service_account
import google.auth
from google.auth.transport.requests import AuthorizedSession
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq as gbq




## Gerar um token para acessar a API
    O token para acessar a API da begrowth é fornecido após cadastrar um
    usuário.

    Informar os dados Nome e E-mail nos campos full_name e email.
    
    Após fazer um request, é retornado o usuário e o token para acessar a API.
    Por fim, imprime na tela o json obtido.

In [14]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = {
    'full_name': 'Fabiano Moreira Alves',
    'email': 'fabianomalves@proton.me',
}

access_token = requests.post('https://begrowth.deta.dev/user/', headers=headers, json=json_data)
access_token_json = access_token.json()

print(access_token_json)


{'user': 'fabianomalves@proton.me', 'API Token': 'BG7X43VA2MLH'}


## Separando o usuário e o token para acessar a API
    Como o request retorna um json(dicionário) é preciso separar o dicionário
    e pegar somente a informação do token.
    
    Após isso, imprime na tela somente o token.

In [15]:
# split dictionary into keys and values
keys = []
values = []
items = access_token_json.items()
for item in items:
    keys.append(item[0]), values.append(item[1])
 
# printing keys and values separately
string_acess_token = str(values[-1])
print(string_acess_token)


BG7X43VA2MLH


## Consumir a api, chamando-a e passando o token adquirido
    Concatenando a url https://begrowth.deta.dev/token=access_token, junto do
    token obtido é necessário para acessar os dados da API.

    Para isso, concatenei a url com o token obtido anteriormente e salvei numa
    variável.
    
    Exibindo a url, concatenando com o token obtido.

    Depois disso, salvar a url com o token obtido na variável
    url_dev_with_token

    Imprime a variável na tela

In [16]:
url_dev = "https://begrowth.deta.dev/token="
url_dev_with_token = url_dev + string_acess_token
print(url_dev_with_token)

https://begrowth.deta.dev/token=BG7X43VA2MLH


## Normalizar o json recebido pelo request, inserindo ele num dataframe Pandas
    Como os dados do obtidos pelo requests são um json, 
    foi feita uma "normalização" dos dados e foi inserido num dataframe.

    Processo feito para trabalhar melhor com os dados, principalmente com campo
    address, que contem outro dicionário dentro da estrutura do json.
    
    Exibindo as primeiras linhas do dataframe.

In [17]:
data = json.loads(requests.get(url_dev_with_token).text)
df_json_normalize = pd.json_normalize(data)
df_json_normalize.head()


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,gAAAAABjieY91IeyDJoBhywKgFQYgUypzOeQeu0Jt9-Yji...,-14.232982,-39.857912,Brazil
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,gAAAAABjieY9-JZwc8sPwkdEb8clzdybJhbaudiEtZ0CZ9...,-6.683513,-35.148879,Brazil
3,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,gAAAAABjieY9-JZwc8sPwkdEb8clzdybJhbaudiEtZ0CZ9...,-6.683513,-35.148879,Brazil
4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,gAAAAABjieY9nowJ0vS-c0ecOFTv0I2X2QnpE4uPcRDEg5...,-5.017543,-42.511724,Brazil


## Procurando linhas duplicadas:
    Ao exibir as primeiras linhas do dataframe, já exibiam algumas linhas
    duplicadas. 
    
    Filtrando os duplicados pelo id dos usuários e retornando 
    todos os duplicados.

In [18]:
duplicate_rows = df_json_normalize[df_json_normalize.duplicated(['id', ])]
print(duplicate_rows)


       id first_name    last_name                         email  gender  \
3       3  Priscilla     Prickett           pprickett2@webs.com  Female   
42     25       Flin  Friedenbach       ffriedenbacho@google.pl    Male   
64     62    Ermanno       Golson        egolson1p@so-net.ne.jp    Male   
82     51     Archer       Moreno            amoreno1e@yale.edu    Male   
109    91        Ara     Carrabot     acarrabot2i@wordpress.org  Female   
195   106      Parke       Wooles       pwooles2x@google.com.au    Male   
256   233   Veronika         Skae            vskae6g@meetup.com  Female   
273   266      Weber       Priest            wpriest7d@java.com    Male   
310   273     Trevor     Yosevitz      tyosevitz7k@redcross.org    Male   
360   336    Vanessa       Allsep        vallsep9b@amazon.co.jp  Female   
429   419     Mortie         Lots             mlotsbm@imgur.com    Male   
478   413    Tiphany     Endricci          tendriccibg@cnet.com  Female   
520   497     Ramsey     

## Removendo as linhas duplicadas
    Dando um drop nas linhas duplicadas, filtrando pelo 'id' e salvando
    em outro dataframe.

    Depois exibe somente as linhas distintas.

In [19]:
df_distinct_id_rows = df_json_normalize.drop_duplicates(subset=['id'])
print(df_distinct_id_rows)


        id   first_name    last_name                         email  gender  \
0        1       Kellen      Cowherd            kcowherd0@nasa.gov    Male   
1        2     Josefina      Swalowe        jswalowe1@slashdot.org  Female   
2        3    Priscilla     Prickett           pprickett2@webs.com  Female   
4        4        Leroi     Spinello          lspinello3@google.fr    Male   
5        5     Bellanca      Laycock       blaycock4@amazonaws.com  Female   
...    ...          ...          ...                           ...     ...   
1014   996        Bride  Grzegorczyk     bgrzegorczykrn@flavors.me  Female   
1015   997        Boote       Downse  bdownsero@huffingtonpost.com    Male   
1017   998       Cindie   Riddlesden    criddlesdenrp@hubpages.com  Female   
1018   999     Roderick      Alfonso      ralfonsorq@quantcast.com    Male   
1019  1000  Konstantine       Perell         kperellrr@blogger.com    Male   

             utm                                               

## Fazendo um reverse geocoding 
    Após normalizar os dados, foram criados 3 novos campos, 
    destrinchando o campo address. 

    Campos address.geo_latitude, address.geo_longitude, address.country.

    Para conseguir a informação do estado, 
    é necessário o processo de reverse geocoding, 
    onde pegamos a latitude e longitude e filtramos as informações que queremos,
    no caso o estado. 
    
    Porém, verifiquei que os nomes dos estados
    apresentam acentos, o que é um problema na hora de fazer os selects.
    
    Para isso, peguei o campo ISO 3166-2, que é um padrão mundial de siglas
    por estados, gerando o padrão BR-UF.
    Como teremos os UFs, será melhor para fazer os selects.

In [23]:
# Create a geocoder object using the Nominatim API
geolocator = Nominatim(user_agent="my_geocoder_state")

# Define a function to reverse geocode the state
def get_state(lat, lng):
  # Use the geocoder object to reverse geocode the coordinates
  location = geolocator.reverse((lat, lng))
  # Extract the state code from the response, using ISO3166-2-lvl4
  state = location.raw['address']['ISO3166-2-lvl4']
  return state

# Apply the function to each row of the DataFrame and store the result in a new column

df_distinct_id_rows.loc[:, ['address_state']] = df_distinct_id_rows.apply(lambda x: get_state(x['address.geo_latitude'], x['address.geo_longitude']), axis=1)



# Display a resulting DataFrame
df_distinct_id_rows.head()



/tmp/ipykernel_8468/2740771092.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_distinct_id_rows.loc[:, ['address_state']] = df_distinct_id_rows.apply(lambda x: get_state(x['address.geo_latitude'], x['address.geo_longitude']), axis=1)


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil,BR-RS
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,gAAAAABjieY91IeyDJoBhywKgFQYgUypzOeQeu0Jt9-Yji...,-14.232982,-39.857912,Brazil,BR-BA
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,gAAAAABjieY9-JZwc8sPwkdEb8clzdybJhbaudiEtZ0CZ9...,-6.683513,-35.148879,Brazil,BR-PB
4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,gAAAAABjieY9nowJ0vS-c0ecOFTv0I2X2QnpE4uPcRDEg5...,-5.017543,-42.511724,Brazil,BR-PI
5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,gAAAAABjieY9ScaiUDpfi3rPkc52PGwUHB8aUJvDYCQBfd...,-21.248487,-45.001375,Brazil,BR-MG


## Salvando o novo dataframe num arquivo csv.
    Como o processo de reverse geocodin demora demais, 
    salvei o dataframe localmente, 
    para não demorar muito, 
    caso fosse necessário rodar novamente o script.

    Consigo trabalhar a partir desse ponto em diante com o arquivo csv,
    caso necessário.

In [25]:
df_distinct_id_rows.to_csv('./data/df_distinct_id_rows.csv', index=False)



# Read csv file

In [48]:
df_distinct_id_rows = pd.read_csv('./data/df_distinct_id_rows.csv')
df_distinct_id_rows.head()

,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil,BR-RS
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,gAAAAABjieY91IeyDJoBhywKgFQYgUypzOeQeu0Jt9-Yji...,-14.232982,-39.857912,Brazil,BR-BA
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,gAAAAABjieY9-JZwc8sPwkdEb8clzdybJhbaudiEtZ0CZ9...,-6.683513,-35.148879,Brazil,BR-PB
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,gAAAAABjieY9nowJ0vS-c0ecOFTv0I2X2QnpE4uPcRDEg5...,-5.017543,-42.511724,Brazil,BR-PI
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,gAAAAABjieY9ScaiUDpfi3rPkc52PGwUHB8aUJvDYCQBfd...,-21.248487,-45.001375,Brazil,BR-MG


## Descriptografar o campo CPF
    O campo CPF está criptografado, pela criptografia Fernet.
    
    Para isso, é necessário usar a chave Fernet, passada por e-mail.
    

In [49]:
fernet = Fernet(b'ekkxXo0uHWRkIbHqHrLS4gaMj2hWTYMJyPTAbi9INGI=')

df_distinct_id_rows['cpf'] = df_distinct_id_rows['cpf'].apply(lambda x: fernet.decrypt(x.encode()))

df_distinct_id_rows.head()



,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,b'91362597210',-28.258844,-53.500999,Brazil,BR-RS
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,b'72137209969',-14.232982,-39.857912,Brazil,BR-BA
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,b'37166330112',-6.683513,-35.148879,Brazil,BR-PB
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,b'86876744655',-5.017543,-42.511724,Brazil,BR-PI
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,b'72461816018',-21.248487,-45.001375,Brazil,BR-MG


## Salvando o arquivo descriptografado num csv

In [50]:

df_distinct_id_rows.to_csv('./data/df_decrypt_cpf.csv', index=False)


## Lendo o arquivo csv, cujo cpf foi descriptografado.

In [51]:
df_decrypt_cpf = pd.read_csv('./data/df_decrypt_cpf.csv' )
df_decrypt_cpf.head()


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,b'91362597210',-28.258844,-53.500999,Brazil,BR-RS
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,b'72137209969',-14.232982,-39.857912,Brazil,BR-BA
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,b'37166330112',-6.683513,-35.148879,Brazil,BR-PB
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,b'86876744655',-5.017543,-42.511724,Brazil,BR-PI
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,b'72461816018',-21.248487,-45.001375,Brazil,BR-MG


## Formatando a coluna CPF
    Após descriptografar o cpf, a coluna ficou um b de byte, 
    oriundo da chave de descriptografar.

    Fazendo a remoção das informações que são desnecessárias.

In [52]:
df_decrypt_cpf['cpf'] = df_decrypt_cpf['cpf'].apply(lambda x: x[2: -1])
df_decrypt_cpf.head()


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,BR-RS
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,BR-BA
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,BR-PB
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,BR-PI
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,BR-MG


## Salvando o arquivo descriptografado e formatado num csv

In [54]:
df_decrypt_cpf['cpf'] = df_decrypt_cpf['cpf'].astype(str)
df_decrypt_cpf.to_csv('./data/df_decrypt_formatting_cpf.csv', index=False)


## Lendo o arquivo csv, descriptografado e formatado e transformando num df

In [55]:
df_decrypt_formatting_cpf = pd.read_csv('data/df_decrypt_formatting_cpf.csv', dtype={'cpf': str})
print(df_decrypt_formatting_cpf.head(20))


    id first_name     last_name                        email  gender  \
0    1     Kellen       Cowherd           kcowherd0@nasa.gov    Male   
1    2   Josefina       Swalowe       jswalowe1@slashdot.org  Female   
2    3  Priscilla      Prickett          pprickett2@webs.com  Female   
3    4      Leroi      Spinello         lspinello3@google.fr    Male   
4    5   Bellanca       Laycock      blaycock4@amazonaws.com  Female   
5    6     Talyah    Brokenshaw         tbrokenshaw5@icq.com  Female   
6    7        Del          Flea            dflea6@reddit.com  Female   
7    8      Avrom          Larn      alarn7@reverbnation.com    Male   
8    9     Hilary         Roper    hroper8@sciencedirect.com    Male   
9   10    Randene       Gilders          rgilders9@google.pl  Female   
10  11     Monica       Fraczak        mfraczaka@state.tx.us  Female   
11  12    Dalenna        Willan           dwillanb@google.pl  Female   
12  13  Clayborne     Charteris       ccharterisc@tripod.com    

## Removendo o prefixo "BR-" da coluna address_state
    Para deixar a coluna mais clean, 
    manter somente as UFs no campo address_state

In [56]:
df_decrypt_formatting_cpf["address_state"] = df_decrypt_formatting_cpf["address_state"].str.replace("BR-", "")
df_decrypt_formatting_cpf.head()


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,RS
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,BA
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,PB
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,PI
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,MG


## Inserindo colunas dt_insert e candidate_name
    Inserindo as colunas dt_insert e candidate_name
    conforme solicitado e prenchendo com um timestamp e o nome.

In [57]:
# Create a variable now, with the actual timestamp
now = datetime.now()

# Create new columns
df_decrypt_formatting_cpf = df_decrypt_formatting_cpf.assign(
    dt_insert=now,
    candidate_name='Fabiano Moreira Alves'
)

df_decrypt_formatting_cpf.head()


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state,dt_insert,candidate_name
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,RS,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,BA,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,PB,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,PI,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,MG,2022-12-30 10:25:41.200942,Fabiano Moreira Alves


## Alterar os nomes das colunas, inserindo um '_' ao invés de '.'
    Removendo os pontos dos nomes das colunas e trocando por underscores.
    Processo necessário para inseir os dados no Google BigQuery.


In [58]:
df_decrypt_formatting_cpf = df_decrypt_formatting_cpf.rename(columns=lambda x: x.replace('.', '_'))

df_decrypt_formatting_cpf.head()


,id,first_name,last_name,email,gender,utm,cpf,address_geo_latitude,address_geo_longitude,address_country,address_state,dt_insert,candidate_name
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,RS,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,BA,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,PB,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,PI,2022-12-30 10:25:41.200942,Fabiano Moreira Alves
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,MG,2022-12-30 10:25:41.200942,Fabiano Moreira Alves


## Salvando o dataframe após as transformações.

In [59]:
df_decrypt_formatting_cpf['cpf'] = df_decrypt_formatting_cpf['cpf'].astype(str)
df_decrypt_formatting_cpf.to_csv('./data/bg_data_enginner_test_fabiano.csv', index=False)



## Conectar no big query e fazer um select
### Tabela: bg_users.brazilian_state

In [63]:

project_id = 'begrowth-user-api-demo'
dataset_id = 'bg_users'

# Set the path to your service account file
service_account_file = '/home/fabiano/Projects/data_engineer_test/svc-data-engineer-test.json'

# Load the service account credentials from the file
credentials = service_account.Credentials.from_service_account_file(service_account_file)

# Create a BigQuery client using your service account credentials
client = bigquery.Client(credentials=credentials, project=project_id)

sql = """
SELECT 
    *
FROM 
begrowth-user-api-demo.bg_users.brazilian_state
"""
df_begrowth_user_api_demo_bg_users_brazilian_state = gbq.read_gbq(sql, project_id = project_id, credentials=credentials, progress_bar_type=None)

print(df_begrowth_user_api_demo_bg_users_brazilian_state)





                     uf initials               region
0     Rio Grande do Sul       RS           Regiao Sul
1        Santa Catarina       SC           Regiao Sul
2                Parana       PR           Regiao Sul
3                 Amapa       AP         Regiao Norte
4              Amazonas       AM         Regiao Norte
5               Roraima       RR         Regiao Norte
6                  Para       PA         Regiao Norte
7              Rondônia       RO         Regiao Norte
8             Tocantins       TO         Regiao Norte
9                  Acre       AC         Regiao Norte
10       Espirito Santo       ES       Regiao Sudeste
11         Minas Gerais       MG       Regiao Sudeste
12       Rio de Janeiro       RJ       Regiao Sudeste
13            Sao Paulo       SP       Regiao Sudeste
14              Alagoas       AL      Regiao Nordeste
15                Ceara       CE      Regiao Nordeste
16  Rio Grande do Norte       RN      Regiao Nordeste
17              Paraiba     

## Salvar o resultado do select num arquivo cvs
    Salvando o conteúdo da tabela 
    bg_users.brazilian_state
    para fazer consultas sql.

In [67]:
df_begrowth_user_api_demo_bg_users_brazilian_state.to_csv('./data/begrowth_user_api_demo_bg_users_brazilian_state.csv', index=False)


## Conectar no big query e fazer um select
### Tabela: begrowth-user-api-demo.bg_users.cpf_state

In [68]:
project_id = 'begrowth-user-api-demo'
dataset_id = 'bg_users'

# Set the path to your service account file
service_account_file = '/home/fabiano/Projects/data_engineer_test/svc-data-engineer-test.json'

# Load the service account credentials from the file
credentials = service_account.Credentials.from_service_account_file(service_account_file)

# Create a BigQuery client using your service account credentials
client = bigquery.Client(credentials=credentials, project=project_id)

sql = """
SELECT 
    *
FROM 
begrowth-user-api-demo.bg_users.cpf_state
"""
df_begrowth_user_api_demo_bg_users_cpf_state = gbq.read_gbq(sql, project_id = project_id, credentials=credentials, progress_bar_type=None)

print(df_begrowth_user_api_demo_bg_users_cpf_state)


   id                      uf
0   7                  ES, RJ
1   9                  PR, SC
2   2  AC, AM, AP, PA, RO, RR
3   6                      MG
4   8                      SP
5   4          AL, PB, PE, RN
6   3              CE, MA, PI
7   0                      RS
8   5                  BA, SE
9   1      DF, GO, MS, MT, TO


## Salvar o resultado do select num arquivo cvs
    Salvando o conteúdo da tabela 
    begrowth-user-api-demo.bg_users.cpf_state
    para fazer consultas sql.

In [69]:
df_begrowth_user_api_demo_bg_users_cpf_state.to_csv('./data/begrowth_user_api_demo_bg_users_cpf_state.csv', index=False)


## Inserir esse dataframe dentro de uma tabela no BigQuery

In [72]:
df = pd.read_csv('./data/bg_data_enginner_test_fabiano.csv', dtype={'cpf': str})




# Set the path to your service account file
service_account_file = '/home/fabiano/Projects/data_engineer_test/svc-data-engineer-test.json'

# Load the service account credentials from the file
credentials = service_account.Credentials.from_service_account_file(service_account_file)

# Create a BigQuery client using your service account credentials
client = bigquery.Client(credentials=credentials, project=project_id)

project_id = 'begrowth-user-api-demo'
dataset_id = 'bg_users'
table_id = 'bg_data_enginner_test_fabiano_moreira_alves'


gbq.to_gbq(df, f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists = 'replace', credentials=credentials)

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/begrowth-user-api-demo/datasets/bg_users/tables/bg_data_enginner_test_fabiano_moreira_alves?prettyPrint=false: Access Denied: Table begrowth-user-api-demo:bg_users.bg_data_enginner_test_fabiano_moreira_alves: Permission bigquery.tables.get denied on table begrowth-user-api-demo:bg_users.bg_data_enginner_test_fabiano_moreira_alves (or it may not exist).